In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
from scipy.optimize import minimize
from scipy.optimize import minimize_scalar
%matplotlib inline
import matplotlib.pyplot as plt

#import models
from sklearn.linear_model import LinearRegression
from sklearn.neural_network import MLPRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn import linear_model
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
#from xgboost import XGBRegressor

from sklearn import model_selection
from sklearn.metrics import explained_variance_score
# from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import mean_squared_error
from tabulate import _table_formats, tabulate
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold

from sklearn import metrics

In [2]:
# datas = './MyData.xlsx'
datas = '/Users/pkrok/OneDrive/Python/Giannakopoulos/MyData.xlsx'
# datas = '/Users/panagiotiskrokidas/OneDrive/Python/Giannakopoulos/MyData.xlsx'

In [3]:
pd.set_option("display.max_columns", None)
df=pd.read_excel(datas)
df.head(2)
df['logD'] = np.log10(df['diffusivity'])
# df = df.dropna()
# df[df.gas == 'propylene']['logD'].max()
df[df.gas == 'H2']['logD'].count()

62

# Keep only the columns I need

In [4]:
# df2=df[[ 'aperture', 'size - van der Waals (Å)','mass', 'ascentricF', 'logD', 'size - kinetic diameter (Å)', 'gas', 'apertureAtom_e', 'ionicRad']]
df2=df[[ 'aperture', 'size - van der Waals (Å)','mass', 'ascentricF', 'logD', 'size - kinetic diameter (Å)', 'ionicRad', 
       'Μ-N_lff', 'Μ-N_kFF', 'ionicRad', 'MetalCharge',
       'apertureAtom_σ', 'apertureAtom_e', 'linker_length1', 'linker_length2',
       'linker_length3', 'linker_mass1', 'linker_mass2', 'linker_mass3',
       'func1_length', 'func2_length', 'func3_length', 'func1_mass',  
       'func2_mass', 'func3_mass', 'func1_charge', 'func2_charge',
       'func3_charge',]]

In [5]:
df2=df2.rename(columns={'size - van der Waals (Å)':'diameter', 'size - kinetic diameter (Å)':'kdiameter', 'apertureAtom_e':'e' })

In [6]:
df2 = df2.dropna()
df2=df2.reset_index(drop=True)
df2

,aperture,diameter,mass,ascentricF,logD,kdiameter,ionicRad,Μ-N_lff,Μ-N_kFF,ionicRad,MetalCharge,apertureAtom_σ,e,linker_length1,linker_length2,linker_length3,linker_mass1,linker_mass2,linker_mass3,func1_length,func2_length,func3_length,func1_mass,func2_mass,func3_mass,func1_charge,func2_charge,func3_charge
0,3.92,2.66,4.002,-0.390,-7.221849,2.60,92,2.257,45354.60,92,1.1901,0.25,0.0627,4.438,4.438,4.438,81.000,81.000,81.000,3.78,3.78,3.78,15.0,15.0,15.0,-0.1001,-0.1001,-0.1001
1,3.92,2.76,2.010,-0.217,-7.301030,2.89,92,2.257,45354.60,92,1.1901,0.25,0.0627,4.438,4.438,4.438,81.000,81.000,81.000,3.78,3.78,3.78,15.0,15.0,15.0,-0.1001,-0.1001,-0.1001
2,3.92,2.94,31.999,0.022,-8.000000,3.46,92,2.257,45354.60,92,1.1901,0.25,0.0627,4.438,4.438,4.438,81.000,81.000,81.000,3.78,3.78,3.78,15.0,15.0,15.0,-0.1001,-0.1001,-0.1001
3,3.92,3.24,44.010,0.225,-8.397940,3.30,92,2.257,45354.60,92,1.1901,0.25,0.0627,4.438,4.438,4.438,81.000,81.000,81.000,3.78,3.78,3.78,15.0,15.0,15.0,-0.1001,-0.1001,-0.1001
4,3.92,3.13,28.000,0.037,-8.301030,3.64,92,2.257,45354.60,92,1.1901,0.25,0.0627,4.438,4.438,4.438,81.000,81.000,81.000,3.78,3.78,3.78,15.0,15.0,15.0,-0.1001,-0.1001,-0.1001
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
593,2.95,2.76,2.010,-0.217,-7.455932,2.89,71,2.139,38994.88,71,1.9871,0.34,1.2552,6.410,6.410,6.410,134.906,134.906,134.906,3.78,3.78,3.78,15.0,15.0,15.0,-0.2710,-0.2710,-0.2710
594,2.95,2.94,31.999,0.022,-9.180456,3.46,71,2.139,38994.88,71,1.9871,0.34,1.2552,6.410,6.410,6.410,134.906,134.906,134.906,3.78,3.78,3.78,15.0,15.0,15.0,-0.2710,-0.2710,-0.2710
595,2.95,3.24,44.010,0.225,-9.096910,3.30,71,2.139,38994.88,71,1.9871,0.34,1.2552,6.410,6.410,6.410,134.906,134.906,134.906,3.78,3.78,3.78,15.0,15.0,15.0,-0.2710,-0.2710,-0.2710
596,2.95,3.13,28.000,0.037,-9.638272,3.64,71,2.139,38994.88,71,1.9871,0.34,1.2552,6.410,6.410,6.410,134.906,134.906,134.906,3.78,3.78,3.78,15.0,15.0,15.0,-0.2710,-0.2710,-0.2710


# Use only van der Waals

In [7]:
x_ar = np.array(df2[['aperture', 'diameter']])
# x_ar = np.asanyarray(df2[['aperture', 'diameter','mass', 'ascentricF', 'kdiameter', 'ionicRad']])
y_ar = np.array(df2[['logD']])

x1 = (df2[['aperture', 'diameter']])
# x1 = (df2[['aperture']])
y1 = (df2[['logD']])

In [8]:
# x_ar.shape
# x_ar.ndim
# x_ar

In [9]:
# data = x_ar
# len(data.shape)

## Optimizer

In [10]:
regr = linear_model.LinearRegression()
tree=DecisionTreeRegressor(max_depth=5,random_state=1)
print(regr)

LinearRegression()


In [11]:
def fitness(X, *args): 
    if X[0] < 5:
        return 7
    else:
        return -1.0/X[0]+ X[1]
bounds=[(-5,100), (-50, 50)]

In [12]:
res = minimize(fitness, np.array([1.0, 5.0]), bounds=bounds, method='L-BFGS-B')

In [13]:
print (str(res))


      fun: 7
 hess_inv: <2x2 LbfgsInvHessProduct with dtype=float64>
      jac: array([0., 0.])
  message: 'CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL'
     nfev: 3
      nit: 0
     njev: 1
   status: 0
  success: True
        x: array([1., 5.])


In [14]:
x_train, x_test, y_train, y_test = train_test_split(x_ar, y_ar, test_size=0.3, random_state=None)

regression=regr.fit(x_train, y_train)

regression.predict([[1.0, 2.0]])

[[-17.2]]

[[-17.2]]

In [15]:
initialParameterValues = x_ar[0]
print(initialParameterValues)
# bounds = [(np.min(x_ar[:, 0]), np.max(x_ar[:, 0])), (np.min(x_ar[:, 1]), np.max(x_ar[:, 1]))]
bounds = [(np.min(x_ar[:, 0]), np.max(x_ar[:, 0])), (3.2, 3.25 )]
print(str(bounds))

# regression.predict([initialParameterValues])
def fitness(PARAMS, *args):
    floatResult = regression.predict([PARAMS])[0][0]
    # print(floatResult)
    # floatResult=floatResult[0]
    return abs(floatResult - -12.0)

res = minimize(fitness, initialParameterValues, bounds=bounds, method='L-BFGS-B', tol=1e-6, options={'eps':1e-10, 
                                                                                'maxiter': 100,
                                                                                })
# res = minimize(fitness, initialParameterValues, bounds=bounds, method='L-BFGS-B', tol=1e-4)
print(str(res))

[3.92 2.66]
[(1.96, 3.92), (3.2, 3.25)]
      fun: 1.3356515893292453e-09
 hess_inv: <2x2 LbfgsInvHessProduct with dtype=float64>
      jac: array([ 5.95210942, -6.78447464])
  message: 'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 141
      nit: 5
     njev: 47
   status: 0
  success: True
        x: array([2.97176976, 3.22395355])


## Add more descriptors

In [57]:
# x_ar = np.array(df2[['aperture', 'diameter']])
# x_ar = np.asanyarray(df2[['aperture', 'diameter', 'ionicRad', 'linker_length1', 'linker_length2',
#        'linker_length3', 'linker_mass1', 'linker_mass2', 'linker_mass3',
#        'func1_length', 'func2_length', 'func3_length', 'func1_mass',
#        'func2_mass', 'func3_mass','e' ]])

x_ar = np.asanyarray(df2[['aperture', 'diameter', 'linker_length1', 'linker_length2', 'linker_length3']])

y_ar = np.array(df2[['logD']])

In [58]:
x_train, x_test, y_train, y_test = train_test_split(x_ar, y_ar, test_size=0.3, random_state=None)
regression=regr.fit(x_train, y_train)

In [59]:
initialParameterValues = x_ar[0]
print(initialParameterValues)
# bounds = [(np.min(x_ar[:, 0]), np.max(x_ar[:, 0])), (np.min(x_ar[:, 1]), np.max(x_ar[:, 1]))]
bounds = [(np.min(x_ar[:, 0]), np.max(x_ar[:, 0])), 
          (3.2, 3.25),
         (np.min(x_ar[:, 2]), np.max(x_ar[:, 2])),
         (np.min(x_ar[:, 2]), np.max(x_ar[:, 3])),
         (np.min(x_ar[:, 2]), np.max(x_ar[:, 4]))]

print(str(bounds))

regression.predict([initialParameterValues])
def fitness(PARAMS, *args):
#     floatResult = regression.predict([PARAMS])[0][0]
    floatResult = regression.predict([PARAMS])[0][0]
    return abs(floatResult - -11.0)

res = minimize(fitness, initialParameterValues, bounds=bounds, method='L-BFGS-B')
print(str(res))

[3.92  2.66  4.438 4.438 4.438]
[(1.96, 3.92), (3.2, 3.25), (3.66, 6.41), (3.66, 6.41), (3.66, 6.41)]
      fun: 2.0028098290936214e-08
 hess_inv: <5x5 LbfgsInvHessProduct with dtype=float64>
      jac: array([ 1.9285377 ,  6.96389013,  0.07950067,  0.07950067, -0.35561918])
  message: 'ABNORMAL_TERMINATION_IN_LNSRCH'
     nfev: 312
      nit: 4
     njev: 52
   status: 2
  success: False
        x: array([3.1763747 , 3.21897003, 4.46872883, 4.46872883, 4.30054481])


In [60]:
print(str(bounds))

[(1.96, 3.92), (3.2, 3.25), (3.66, 6.41), (3.66, 6.41), (3.66, 6.41)]


In [61]:
x_ar[0]

array([3.92 , 2.66 , 4.438, 4.438, 4.438])

In [62]:
# df2[['aperture', 'diameter', 'linker_length1', 'linker_mass2', 'linker_mass3']]

## Decision Tree Regressor

In [63]:
x_ar = np.array(df2[['aperture', 'diameter']])
# x_ar = np.asanyarray(df2[['aperture', 'diameter', 'ionicRad', 'linker_length1', 'linker_length2',
#        'linker_length3', 'linker_mass1', 'linker_mass2', 'linker_mass3',
#        'func1_length', 'func2_length', 'func3_length', 'func1_mass',
#        'func2_mass', 'func3_mass','e' ]])

# x_ar = np.asanyarray(df2[['aperture', 'diameter', 'linker_length1', 'linker_length2', 'linker_length3']])

y_ar = np.array(df2[['logD']])

In [64]:
x_train, x_test, y_train, y_test = train_test_split(x_ar, y_ar, test_size=0.3, random_state=None)
regression=tree.fit(x_train, y_train)

In [71]:
initialParameterValues = x_ar[0]
print(initialParameterValues)
# bounds = [(np.min(x_ar[:, 0]), np.max(x_ar[:, 0])), (np.min(x_ar[:, 1]), np.max(x_ar[:, 1]))]

bounds = [(np.min(x_ar[:, 0]), np.max(x_ar[:, 0])), (3.2, 3.25 )]

print(str(bounds))

# regression.predict([initialParameterValues])
def fitness(PARAMS, *args):
    floatResult = regression.predict([PARAMS])[0]
    print(floatResult)
    return abs(floatResult - -12.0)

res = minimize(fitness, initialParameterValues, bounds=bounds, method='L-BFGS-B', tol=1e-8, options={'eps':1e-10, 
                                                                                'maxiter': 100,
                                                                                })

# res = minimize(fitness, initialParameterValues, method='TNC', bounds=bounds)

# res = minimize(fitness, initialParameterValues, bounds=bounds, method='L-BFGS-B', tol=1e-4)
print(str(res))

[3.92 2.66]
[(1.96, 3.92), (3.2, 3.25)]
-8.441307735295183
-8.441307735295183
-8.441307735295183
      fun: 3.558692264704817
 hess_inv: <2x2 LbfgsInvHessProduct with dtype=float64>
      jac: array([-0.,  0.])
  message: 'CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL'
     nfev: 3
      nit: 0
     njev: 1
   status: 0
  success: True
        x: array([3.92, 3.2 ])


## Decision Tree - more descriptors

In [517]:
# x_ar = np.array(df2[['aperture', 'diameter']])
# x_ar = np.asanyarray(df2[['aperture', 'diameter', 'ionicRad', 'linker_length1', 'linker_length2',
#        'linker_length3', 'linker_mass1', 'linker_mass2', 'linker_mass3',
#        'func1_length', 'func2_length', 'func3_length', 'func1_mass',
#        'func2_mass', 'func3_mass','e' ]])

x_ar = np.asanyarray(df2[['aperture', 'diameter', 'linker_length1', 'linker_length2', 'linker_length3']])

y_ar = np.array(df2[['logD']])

In [518]:
x_train, x_test, y_train, y_test = train_test_split(x_ar, y_ar, test_size=0.3, random_state=None)
regression=tree.fit(x_train, y_train)

In [519]:
initialParameterValues = x_ar[0]
print(initialParameterValues)
# bounds = [(np.min(x_ar[:, 0]), np.max(x_ar[:, 0])), (np.min(x_ar[:, 1]), np.max(x_ar[:, 1]))]
bounds = [(np.min(x_ar[:, 0]), np.max(x_ar[:, 0])), 
          (3.2, 4.25),
         (np.min(x_ar[:, 2]), np.max(x_ar[:, 2])),
         (np.min(x_ar[:, 2]), np.max(x_ar[:, 3])),
         (np.min(x_ar[:, 2]), np.max(x_ar[:, 4]))]

print(str(bounds))

regression.predict([initialParameterValues])
def fitness(PARAMS, *args):
#     floatResult = regression.predict([PARAMS])[0][0]
    floatResult = tree.predict([PARAMS])[0]
    return abs(floatResult - -16.0)

res = minimize(fitness, initialParameterValues, bounds=bounds, method='L-BFGS-B', tol=1e-9, options={'eps':1e-12, 
                                                                                'maxiter': 15500,
                                                                                })
print(str(res))

[3.92  2.66  4.438 4.438 4.438]
[(1.96, 3.92), (3.2, 4.25), (3.66, 6.41), (3.66, 6.41), (3.66, 6.41)]
      fun: 7.4926125506947
 hess_inv: <5x5 LbfgsInvHessProduct with dtype=float64>
      jac: array([-0.,  0.,  0.,  0.,  0.])
  message: 'CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL'
     nfev: 6
      nit: 0
     njev: 1
   status: 0
  success: True
        x: array([3.92 , 3.2  , 4.438, 4.438, 4.438])
